In [1]:
!pip install transformers

In [2]:
# thu vien chinh sua cac loi unicode
!pip install ftfy

In [4]:
import pandas as pd
import numpy as np
import os
from pprint import pprint
import string    
import random
import json
import spacy
from spacy import displacy
from transformers import BertTokenizer, BertForTokenClassification
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification


In [4]:
# connect ggdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:

#save_model_address = '/content/drive/MyDrive/LUẬN VĂN-K18_CQ/02. Bình-Ngọc/Data/Dataset/conll/b_simp'
#save_model_address = '/content'

save_model_address = '/home/tyeson/Desktop/ML_NER/NERModel_config'

#save_model = BertForTokenClassification.from_pretrained(save_model_address, num_labels=20)
#tokenizer = BertTokenizer.from_pretrained(save_model_address,do_lower_case=True)

save_model = AutoModelForTokenClassification.from_pretrained(save_model_address)
tokenizer = AutoTokenizer.from_pretrained(save_model_address, do_lower_case=True, model_max_length=256)

nlp = pipeline("ner", model=save_model, tokenizer=tokenizer, aggregation_strategy='simple',ignore_labels =['X','O'])


In [6]:
# test cau dai
orig_string = '''Learn and Master Software Testing Quickly from the experts - GUARANTEED! THE IN-DEPTH SOFTWARE TESTING TRAINING - By SoftwareTestingHelp Team. "TOP STUDENT PICK" on Udemy in the Software Testing category! 26+ hours of HD content. Value for money! DON'T settle for other basic courses of less thanhours! Few Student reviews from hundreds ofstar reviews: "The course is an eye opener into the world of IT. Theophilus. "Money well spent, excellent delivery. Very informative and practical. Would highly recommend to anyone interested in pursuing software testing as a career. Olanrewaju. "Truly the best software testing training I have come across both in dept and in substance. Kingsley. "This is really "The Best Software Training Course". I hardly know anything regarding testing, instructor had taken utmost care in providing the knowledge starting from basics, the terminology etc...I am very much satisfied with this course. I strongly recommend this course. Vijaya. "Great tutorials ..in detail ...learned a lot ...must see tutorial for all testers. Masud. "The instructor is just a perfect lecturer! Entire course is very informative and useful for software testers as beginners with a lot of practical examples. Who wants to understand principles of testing and main techniques of it - enroll in this course. Oleksii. "The instructor according to me.....God has gifted her a real talent to be one of the best tutors in this world. Biju. Introducing the Most Practical, Precise and Inexpensive Software Testing Course. It is going to include everything there is to know for you to become a perfect Software Tester. This software testing QA training course is designed by working professionals in a way that, course it will progress from introducing you to the basics of software testing to advanced topics like Software configuration management, creating a test plan, test estimations etc along with introduction and familiarity with Automation testing and test management tools like QTP (intro), QC, JIRA, and Bugzilla. Course Benefits: Syllabus: We came up with a unique list of topics that will help you gradually work your way into the testing world. Practice sessions: Assignments in a way that you will get to apply the theory you learnt immediately. Video sessions of Instructor led live training sessions. Practical learning experienc e with live project work and examples. Support: Our Team is going to be available to you via email or the website for you to reach out to us. Over Lectures and more than+ hours of HD content! Learn Software Testing and Automation basics from a professional trainer from your own desk. Information packed practical training starting from basics to advanced testing techniques. Best suitable for beginners to advanced level users and who learn faster when demonstrated. Get â€œCertificate of completion. LIVE PROJECT End to End Software Testing Training Included. Learn Software Testing and Automation basics from a professional trainer from your own desk. Information packed practical training starting from basics to advanced testing techniques. Best suitable for beginners to advanced level users and who learn faster when demonstrated. Course content designed by considering current software testing technology and the job market. Practical assignments at the end of every session. Practical learning experience with live project work and examples. Lifetime enrollment - Pay one time fee and access video training sessions as many times as you want. Resume Preparation Guidance for Testers Included. Software Testing Interview Questions and Preparation Tips Included. Download Real Software Testing Templates like Test Plan, Test Cases and other important Templates. Software Testing Certification Guidance. Learn Test Management Tools like JIRA, and Bugzilla. Get all future course updates free!'''
#results = nlp(sentences)
#results
#len(tokenizer.tokenize(sentences, truncation=True))
list_of_lines = []
max_length = 256*4
while len(orig_string) > max_length:
    line_length = max(orig_string[:max_length].rfind(i) for i in ".!?,")
    list_of_lines.append(orig_string[:line_length])
    orig_string = orig_string[line_length + 1:]
list_of_lines.append(orig_string)
list_of_lines


['Learn and Master Software Testing Quickly from the experts - GUARANTEED! THE IN-DEPTH SOFTWARE TESTING TRAINING - By SoftwareTestingHelp Team. "TOP STUDENT PICK" on Udemy in the Software Testing category! 26+ hours of HD content. Value for money! DON\'T settle for other basic courses of less thanhours! Few Student reviews from hundreds ofstar reviews: "The course is an eye opener into the world of IT. Theophilus. "Money well spent, excellent delivery. Very informative and practical. Would highly recommend to anyone interested in pursuing software testing as a career. Olanrewaju. "Truly the best software testing training I have come across both in dept and in substance. Kingsley. "This is really "The Best Software Training Course". I hardly know anything regarding testing, instructor had taken utmost care in providing the knowledge starting from basics, the terminology etc...I am very much satisfied with this course. I strongly recommend this course. Vijaya. "Great tutorials ..in deta

In [7]:
# fix ky tu bang thu vien fix that for you
import ftfy
results = nlp(ftfy.fix_text(list_of_lines[0]))
results

[{'entity_group': 'KNOW',
  'score': 0.9037936,
  'word': 'software testing',
  'start': 17,
  'end': 33},
 {'entity_group': 'KNOW',
  'score': 0.8402615,
  'word': 'software testing',
  'start': 86,
  'end': 102},
 {'entity_group': 'KNOW',
  'score': 0.8576294,
  'word': 'software testing',
  'start': 543,
  'end': 559},
 {'entity_group': 'KNOW',
  'score': 0.8752375,
  'word': 'software testing',
  'start': 601,
  'end': 617}]

In [8]:
#read cot text tu file csv
#df = pd.read_csv('udemy2.csv')
df = pd.read_csv('/home/tyeson/Desktop/K18/File K18/sample_crawl_dataset/Coursera_DataScience.csv')
print(len(df))
row_len = len(df) - 1
skillwilllearn_text= list(df["SkillWillLearn"])


282


In [9]:
#len(list_txt[1])
#len(tokenizer.tokenize(list_txt[470], truncation=True))

len(skillwilllearn_text[1])
len(tokenizer.tokenize(skillwilllearn_text[row_len], truncation=True))

41

In [11]:
#pprint(results)
res=[]
count=0
for sent in skillwilllearn_text:
  print(count)
  count+=1
  results = nlp(ftfy.fix_text(sent))
  fm=[]
  ents=[]
  for ele in results: # add character indexes
      span = {'start':ele["start"],'end': ele["end"], 
      'labels':[ele["entity_group"]], "score":float(ele["score"]), "text":ele["word"]}
      ents.append({"id":''.join(random.choices(string.ascii_uppercase + string.digits, k=6)),
                  "from_name": "label",
                  "to_name": "text",
                  "type": "labels",
                  "value": span})

  fm={"data":{"text":sent},
      "predictions": [{
          "model_version": "one",
          "result": ents
  }]}
  res.append(fm)

#pprint(res)
with open("/home/tyeson/Desktop/ML_NER/output/coursera_skillwilllearn.json", "w") as outfile:
    json.dump(res, outfile)

print('done')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [13]:
import json
f = open("/home/tyeson/Desktop/ML_NER/output/coursera_skillwilllearn.json")
data = json.load(f)

num = 0

for i in data:
  print(i)
  if num == 3:
    break
  num = num +1
#print(data)
f.close()

{'data': {'text': "Machine learning is the science of getting computers to act without being explicitly programmed. In the past decade, machine learning has given us self-driving cars, practical speech recognition, effective web search, and a vastly improved understanding of the human genome. Machine learning is so pervasive today that you probably use it dozens of times a day without knowing it. Many researchers also think it is the best way to make progress towards human-level AI. In this class, you will learn about the most effective machine learning techniques, and gain practice implementing them and getting them to work for yourself. More importantly, you'll learn about not only the theoretical underpinnings of learning, but also gain the practical know-how needed to quickly and powerfully apply these techniques to new problems. Finally, you'll learn about some of Silicon Valley's best practices in innovation as it pertains to machine learning and AI. This course provides a broad 

In [14]:
#ket qua cuoi cung cua SkillWillLearn

#temp
id = 0
arr = []
arr_1D = []
ls = []
SkillWillLearn_final_ls = []
for i in data:
  for j in i['predictions']:
    for k in j['result']:
      res = k['value']
      #print(res['text'])
      #ls.append(res['text'])
      #print(res['labels'][0])
      arr_1D.append(res['text'])
      arr_1D.append(res['labels'][0])
      arr.append(arr_1D)
      #print(arr)     
      arr_1D = []


  #print(id)
  #id = id + 1
  #print(arr)
  arr = list(set([tuple(arr) for arr in arr]))
  #final_ls.append(arr)
  SkillWillLearn_final_ls.append(arr)
  arr_1D = []
  arr = []

print(SkillWillLearn_final_ls)

[[('clustering', 'KNOW'), ('statistical pattern recognition', 'KNOW'), ('support vector machines', 'KNOW'), ('supervised learning', 'KNOW'), ('machine learning', 'KNOW'), ('##s', 'KNOW'), ('dimensionality reduction', 'KNOW'), ('unsupervised learning', 'KNOW'), ('recommender systems', 'KNOW'), ('algorithms', 'KNOW'), ('deep', 'KNOW'), ('ai', 'KNOW'), ('datamining', 'KNOW'), ('neural networks', 'KNOW')], [('charts', 'KNOW'), ('excel', 'TOOL'), ('visualization', 'KNOW'), ('visualizations', 'KNOW'), ('big data', 'KNOW'), ('waterfall charts', 'KNOW'), ('data visualization', 'KNOW'), ('sunburst diagrams', 'KNOW'), ("excel's", 'TOOL')], [('gis', 'KNOW'), ('analyze map data', 'KNOW'), ('analysis flowcharts', 'KNOW'), ('modelbuilder', 'TOOL'), ('vector data', 'KNOW')], [('python', 'LANG'), ('data structures', 'KNOW')], [('machine learning', 'KNOW'), ('deep', 'KNOW'), ('neural networks ;', 'KNOW'), ('ai', 'KNOW'), ('deep learning', 'KNOW'), ("neural network's", 'KNOW'), ('deep learning ;', 'KNOW

In [15]:
row_len = len(df) - 1
skillrequirement_text= list(df["skillrequirement"])

skillrequirement_text=['None' if x is np.nan else x for x in skillrequirement_text]
print(skillrequirement_text)

['None', 'We expect that you have some background with Excel. You can move around the interface and create basic formulas.', 'None', 'None', 'None', 'None', 'None', 'No prior experience with spreadsheets or data analytics is required. All you need is high-school level math and a curiosity about how things work. ', 'None', 'None', 'High school algebra', 'Some prior experience using data in a business setting is helpful.', 'F\u200bamiliarity with the R programming language', 'None', 'None', 'None', 'While there are no prerequisites, prior experience with, or knowledge of, health, healthcare, technology, and statistics are helpful.', 'None', 'None', 'None', 'None', 'Sequence in calculus up through Calculus II (preferably multivariate calculus) and some programming experience in R.', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'Computer and IT literacy. ', 'None', 'None', 'None', 'None', 'None', 'Learners should have completed the previou

In [16]:
len(skillrequirement_text[1])
len(tokenizer.tokenize(skillrequirement_text[row_len], truncation=True))

1

In [18]:
#pprint(results)
res=[]
count=0
for sent in skillrequirement_text:
  print(count)
  count+=1
  results = nlp(ftfy.fix_text(sent))
  fm=[]
  ents=[]
  for ele in results: # add character indexes
      span = {'start':ele["start"],'end': ele["end"], 
      'labels':[ele["entity_group"]], "score":float(ele["score"]), "text":ele["word"]}
      ents.append({"id":''.join(random.choices(string.ascii_uppercase + string.digits, k=6)),
                  "from_name": "label",
                  "to_name": "text",
                  "type": "labels",
                  "value": span})

  fm={"data":{"text":sent},
      "predictions": [{
          "model_version": "one",
          "result": ents
  }]}
  res.append(fm)

#pprint(res)
with open("/home/tyeson/Desktop/ML_NER/output/coursera_skillrequirement.json", "w") as outfile:
    json.dump(res, outfile)

print('done')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [19]:
f = open("/home/tyeson/Desktop/ML_NER/output/coursera_skillrequirement.json")
data = json.load(f)

num = 0

for i in data:
  print(i)
  if num == 3:
    break
  num = num +1
#print(data)
f.close()

{'data': {'text': 'None'}, 'predictions': [{'model_version': 'one', 'result': []}]}
{'data': {'text': 'We expect that you have some background with Excel. You can move around the interface and create basic formulas.'}, 'predictions': [{'model_version': 'one', 'result': [{'id': '8G4I3Z', 'from_name': 'label', 'to_name': 'text', 'type': 'labels', 'value': {'start': 45, 'end': 50, 'labels': ['TOOL'], 'score': 0.9761345982551575, 'text': 'excel'}}]}]}
{'data': {'text': 'None'}, 'predictions': [{'model_version': 'one', 'result': []}]}
{'data': {'text': 'None'}, 'predictions': [{'model_version': 'one', 'result': []}]}


In [20]:
#ket qua cuoi cung cua skillrequirement

#temp
id = 0
arr = []
arr_1D = []
ls = []
skillrequirement_final_ls = []
for i in data:
  for j in i['predictions']:
    for k in j['result']:
      res = k['value']
      #print(res['text'])
      #ls.append(res['text'])
      #print(res['labels'][0])
      arr_1D.append(res['text'])
      arr_1D.append(res['labels'][0])
      arr.append(arr_1D)
      #print(arr)     
      arr_1D = []


  #print(id)
  #id = id + 1
  #print(arr)
  arr = list(set([tuple(arr) for arr in arr]))
  #final_ls.append(arr)
  skillrequirement_final_ls.append(arr)
  arr_1D = []
  arr = []

print(skillrequirement_final_ls)

[[], [('excel', 'TOOL')], [], [], [], [], [], [('spreadsheets', 'TOOL'), ('data analytics', 'KNOW')], [], [], [], [], [('r', 'LANG')], [], [], [], [('healthcare', 'KNOW'), ('statistics', 'KNOW')], [], [], [], [], [('r', 'LANG'), ('multivariate calculus', 'KNOW')], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [('python', 'LANG')], [], [], [('python', 'LANG')], [], [], [], [], [], [], [], [], [], [('spreadsheets', 'TOOL'), ('data analytics', 'KNOW')], [('r', 'LANG'), ('multivariate calculus', 'KNOW')], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [('ai', 'KNOW'), ('deep learning', 'KNOW')], [], [('python', 'LANG')], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [('quantitative data', 'KNOW')], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [('spreadsheets', 'TOOL'), ('data analytics', 'KNOW')], [], [('gis', 'KNOW'), ('data analysis', 'KNOW')], [], [], [], [], [], [], [], [('python', 'LANG'), ('data science', 'KN

In [21]:
name_ls = list(df['name'])
link_ls = list(df['link'])
rating_ls = list(df['rating'])
enroll_ls = list(df['enroll'])
instructor_ls = list(df['instructor'])
levelrequirement_ls = list(df['levelrequirement'])
skillrequirement_ls = list(df['skillrequirement'])
SkillWillLearn_ls = list(df['SkillWillLearn'])
SkillGain_ls = list(df['SkillGain'])
Subject_ls = list(df['Subject'])
organization_ls = list(df['organization'])
fee_ls = list(df['fee'])
program_ls = list(df['program'])
RelationInsOrg_ls = list(df['RelationInsOrg'])
Subtitle_ls = list(df['Subtitle'])

levelrequirement_ls=['Beginner' if x is np.nan else x for x in levelrequirement_ls]
Subtitle_ls = [x.split(", ") for x in Subtitle_ls]
SkillGain_ls=['None' if x is np.nan else x for x in SkillGain_ls]
#SkillGain_ls = [x.split() for x in SkillGain_ls]
print(Subtitle_ls)

[['Arabic', 'French', 'Portuguese (European)', 'Chinese (Simplified)', 'Italian', 'Vietnamese', 'German', 'Russian', 'English', 'Hebrew', 'Spanish', 'Hindi', 'Japanese'], ['English'], ['Arabic', 'French', 'Portuguese (European)', 'Italian', 'Vietnamese', 'German', 'Russian', 'English', 'Spanish'], ['Arabic', 'French', 'Portuguese (European)', 'Chinese (Simplified)', 'Italian', 'Vietnamese', 'Korean', 'German', 'Russian', 'English', 'Spanish'], ['Chinese (Traditional)', 'Arabic', 'French', 'Portuguese (European)', 'Chinese (Simplified)', 'Italian', 'Portuguese (Brazilian)', 'Vietnamese', 'Korean', 'German', 'Russian', 'Turkish', 'English', 'Spanish', 'Japanese'], ['Arabic', 'French', 'Portuguese (European)', 'Italian', 'Vietnamese', 'German', 'Russian', 'Turkish', 'English', 'Spanish'], ['English'], ['English'], ['French', 'Portuguese (European)', 'Russian', 'English', 'Spanish'], ['Subtitles: English'], ['Arabic', 'French', 'Portuguese (European)', 'Italian', 'Vietnamese', 'Korean', 'G

In [22]:
course_dict = {}
list_dict = []

for i in range(row_len):
  course_dict['name'] = name_ls[i]
  course_dict['link'] = link_ls[i]
  course_dict['rating'] = rating_ls[i]
  course_dict['instructor'] = instructor_ls[i]
  course_dict['levelrequirement'] = levelrequirement_ls[i]

  course_dict['skillrequirement'] = skillrequirement_final_ls[i]
  
  course_dict['SkillWillLearn'] = SkillWillLearn_final_ls[i]
  course_dict['SkillGain'] = SkillGain_ls[i]
  course_dict['Subject'] = Subject_ls[i]
  course_dict['organization'] = organization_ls[i]
  course_dict['fee'] = fee_ls[i]
  course_dict['program'] = program_ls[i]
  course_dict['RelationInsOrg'] = RelationInsOrg_ls[i]
  course_dict['Subtitle'] = Subtitle_ls[i]
  list_dict.append(course_dict)
  course_dict = {}

for i in (list_dict):
  print(i)

{'name': 'Machine Learning', 'link': 'https://www.coursera.org/learn/machine-learning', 'rating': 4.9, 'instructor': 'Andrew Ng', 'levelrequirement': 'Beginner', 'skillrequirement': [], 'SkillWillLearn': [('clustering', 'KNOW'), ('statistical pattern recognition', 'KNOW'), ('support vector machines', 'KNOW'), ('supervised learning', 'KNOW'), ('machine learning', 'KNOW'), ('##s', 'KNOW'), ('dimensionality reduction', 'KNOW'), ('unsupervised learning', 'KNOW'), ('recommender systems', 'KNOW'), ('algorithms', 'KNOW'), ('deep', 'KNOW'), ('ai', 'KNOW'), ('datamining', 'KNOW'), ('neural networks', 'KNOW')], 'SkillGain': 'Logistic Regression          Artificial Neural Network          Machine Learning (ML) Algorithms          Machine Learning      ', 'Subject': 'Data Science', 'organization': 'Stanford University', 'fee': 'Enroll for Free', 'program': nan, 'RelationInsOrg': 'Andrew Ng work for Stanford University, Coursera, DeepLearning.AI', 'Subtitle': ['Arabic', 'French', 'Portuguese (Europ

In [23]:
for i in list_dict:
  print(i['name'])

Machine Learning
Data Visualization in Excel
Spatial Analysis and Satellite Imagery in a GIS
Python Data Structures
Neural Networks and Deep Learning
What is Data Science? 
Web of Data
Share Data Through the Art of Visualization
Visualization for Data Journalism
Using Data for Geographic Mapping and Forecasting in SAS Visual Analytics
Understanding and Visualizing Data with Python
Tools for Exploratory Data Analysis in Business
Importing Data in the Tidyverse
Visualizing Data in the Tidyverse
Wrangling Data in the Tidyverse
Modeling Data in the Tidyverse
The Data Science of Health Informatics
The Fundamental of Data-Driven Investment
Teaching Impacts of Technology: Data Collection, Use, and Privacy
Survey Data Collection and Analytics Project (Capstone)
Statistics for Data Science with Python
Statistical Inference for Estimation in Data Science
SQL for Data Science with R
SQL for Data Science
SQL for Data Science Capstone Project
Databases and SQL for Data Science with Python
Spatial D

In [24]:
!pip install neo4j
!pip install py2neo


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [25]:
from neo4j import GraphDatabase

In [46]:
graphdb = GraphDatabase.driver(uri='bolt://20.239.83.14:7687', auth = ('graphdw','password'))

print(graphdb)


In [47]:
session = graphdb.session()

q1='MATCH (x) return (x)'
nodes = session.run(q1)

for node in nodes:
  print(node)

ServiceUnavailable: Couldn't connect to 20.239.83.14:7687 (resolved to ()):
Failed to establish connection to ResolvedIPv4Address(('20.239.83.14', 7687)) (reason [Errno 111] Connection refused)